# Bert baseline for POLAR

## Introduction

In this part of the starter notebook, we will take you through the process of all three Subtasks.

## Subtask 1 - Polarization detection

This is a binary classification to determine whether a post contains polarized content (Polarized or Not Polarized).

In [1]:
# !unzip dev_phase.zip

## Imports

In [1]:
import pandas as pd

from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np

import torch

from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from torch.utils.data import Dataset

In [2]:
import wandb

# Disable wandb logging for this script
wandb.init(mode="disabled")

## Data Import

The training data consists of a short text and binary labels

The data is structured as a CSV file with the following fields:
- id: a unique identifier for the sample
- text: a sentence or short text
- polarization:  1 text is polarized, 0 text is not polarized

The data is in all three subtask folders the same but only containing the labels for the specific task.

In [3]:
# Load the training and validation data for subtask 1

from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('./dev_phase/subtask1/train/eng.csv')

# Split into train (80%) and val (20%), stratify if possible
train, val = train_test_split(
    data,
    test_size=0.2,
    random_state=42,
    stratify=data["polarization"] if "polarization" in data.columns else None
)

train.head()

,id,text,polarization
1350,eng_f308ca27dccc22549e50f1042ceb1df8,And where did I say h8 or xenophobia?,0
2605,eng_b1d3f6b9a86d738b0dbe5a5d891f79ef,WOW bad will the RedWAVE bloodbath be for the ...,1
1754,eng_5836d1c4db446e1a501fe30fb1b2615c,"Breitbart is racist trash, for revealing Racis...",1
634,eng_789191ae3e9c3a9c6de6565559820379,"Israeli Bedouins, lacking government protectio...",0
2700,eng_737e402f83d6f36f2023195aecac92df,"As Joe Biden leaves the White House today, I r...",0


# Dataset
-  Create a pytorch class for handling data
-  Wrapping the raw texts and labels into a format that Huggingface’s Trainer can use for training and evaluation

In [4]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
  def __init__(self,texts,labels,tokenizer,max_length =128):
    self.texts=texts
    self.labels=labels
    self.tokenizer= tokenizer
    self.max_length = max_length # Store max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text=self.texts[idx]
    label=self.labels[idx]
    encoding=self.tokenizer(text,truncation=True,padding=False,max_length=self.max_length,return_tensors='pt')

    # Ensure consistent tensor conversion for all items
    item = {key: encoding[key].squeeze() for key in encoding.keys()}
    item['labels'] = torch.tensor(label, dtype=torch.long)
    return item

Now, we'll tokenize the text data and create the datasets using `bert-base-uncased` as the tokenizer.

In [5]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

# Create datasets
train_dataset = PolarizationDataset(train['text'].tolist(), train['polarization'].tolist(), tokenizer)
val_dataset = PolarizationDataset(val['text'].tolist(), val['polarization'].tolist(), tokenizer)

/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Next, we'll load the pre-trained `bert-base-uncased` model for sequence classification. Since this is a binary classification task (Polarized/Not Polarized), we set `num_labels=2`.

In [6]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=2)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, we'll define the training arguments and the evaluation metric. We'll use macro F1 score for evaluation.

In [7]:
# Define metrics function
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

# Define training arguments
training_args = TrainingArguments(
        output_dir=f"./",
        num_train_epochs=3,
        learning_rate=2e-5,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=8,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=100,
        disable_tqdm=False
    )


Finally, we'll initialize the `Trainer` and start training.

In [8]:
# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    data_collator=DataCollatorWithPadding(tokenizer) # Data collator for dynamic padding
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Macro F1 score on validation set: {eval_results['eval_f1_macro']}")

/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG15XFamilyCommandBuffer: 0x36cae3590>
    label = <none> 
    device = <AGXG15SDevice: 0x366818600>
        name = Apple M3 Pro 
    commandQueue = <AGXG15XFamilyCommandQueue: 0x309c8fa00>
        label = <none> 
        device = <AGXG15SDevice: 0x366818600>
            name = Apple M3 Pro 
    retainedReferences = 1


RuntimeError: MPS backend out of memory (MPS allocated: 15.46 GiB, other allocations: 7.14 GiB, max allowed: 22.64 GiB). Tried to allocate 183.00 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
from tqdm import tqdm

In [ ]:
test_dataset = pd.read_csv('./dev_phase/subtask1/dev/eng.csv')
labels = []
probs_list = []
labels = []
for text in tqdm(test_dataset['text']):
    # Run the model
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred_label = logits.argmax(dim=1).cpu().numpy()[0]
        labels.append(pred_label)
        probs_list.append(probabilities)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 169/169 [00:02<00:00, 84.40it/s] 


In [ ]:
# print the results row by row in csv format
for i in range(len(labels)):
    print(f"{test_dataset['id'][i]},{labels[i]}")


arb_67be47e5216d7bee41e17484e619f4e6,1
arb_272322e5b265e177613d685e5619e402,0
arb_d1ec38dd0ec5d7a4fe28ef8317fc96c1,0
arb_fad75310b17c124d98ebc514189ec033,1
arb_95caf70cec5bf00c94c35cf7af2a0ab5,1
arb_ac108c1ecf5071892c61abd253847b15,1
arb_adaaa6d482119e65ce337ee224674e70,1
arb_2794b08cac6cc9394a68c51cfc436243,1
arb_19dd96c989323c9e950a2c3ab9c285be,1
arb_f2bd638d9d9fc7a617130ff2b198b562,1
arb_f992bf7776b854d4f7f8475aebf80f49,1
arb_0b5ac70e86926f5e84cad94028864a37,1
arb_8ababf95f952e2425c2df1033192dac0,0
arb_06cd19aac6cc52e394a22d7d1dd58efc,0
arb_12eeeb8d2fa2d04be2ed9830d5f36ce9,0
arb_5bc23bacf9a161cd0f99719c70681a81,1
arb_9ee7c931ab1ecd655533042d8301f6bb,0
arb_bb7c40559f3a7ca1ecdd7dd7c136198f,0
arb_5d394c0cce56675e2fc36a0590b47ed7,0
arb_0704305e8313650e672563a2d073384f,0
arb_e56b759d14fd70506e01cf971315453d,0
arb_d286a2aac63432acef285a4799041f55,0
arb_5a9f322a530e85cd640f21af5c6bae42,1
arb_133c4737c4e04a8991fc7c219106b4e4,1
arb_f03fd416ab63cbb8f9f92a020fdd46be,1
arb_b8fbf2253ba4b8a83829b

# Subtask 2: Polarization Type Classification
Multi-label classification to identify the target of polarization as one of the following categories: Gender/Sexual, Political, Religious, Racial/Ethnic, or Other.
For this task we will load the data for subtask 2.

In [ ]:
# Load the data
data = pd.read_csv('./dev_phase/subtask2/train/eng.csv')

# Split into train (80%) and val (20%), stratify if possible
train, val = train_test_split(
    data,
    test_size=0.2,
    random_state=42,
)

In [ ]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length # Store max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding=False, max_length=self.max_length, return_tensors='pt')

        # Ensure consistent tensor conversion for all items
        item = {key: encoding[key].squeeze() for key in encoding.keys()}
        # CHANGE THIS LINE: Use torch.float instead of torch.long for multi-label classification
        item['labels'] = torch.tensor(label, dtype=torch.float)
        return item


In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

# Create train and Test dataset for multilabel
train_dataset = PolarizationDataset(train['text'].tolist(), train[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)
val_dataset = PolarizationDataset(val['text'].tolist(), val[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)
dev_dataset = PolarizationDataset(val['text'].tolist(), val[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)


In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=5, problem_type="multi_label_classification") # 5 labels

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define metrics function for multi-label classification
def compute_metrics_multilabel(p):
    # Sigmoid the predictions to get probabilities
    probs = torch.sigmoid(torch.from_numpy(p.predictions))
    # Convert probabilities to predicted labels (0 or 1)
    preds = (probs > 0.5).int().numpy()
    # Compute macro F1 score
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

# Define training arguments
training_args = TrainingArguments(
    output_dir=f"./",
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=100,
    disable_tqdm=False
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_multilabel,  # Use the new metrics function
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Macro F1 score on validation set for Subtask 2: {eval_results['eval_f1_macro']}")

/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.320600,0.294996,0.327454
2,0.242300,0.261739,0.586681
3,0.201000,0.257001,0.610804


/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Macro F1 score on validation set for Subtask 2: 0.6108039208443157


In [ ]:
from tqdm import tqdm
test_dataset = pd.read_csv('./dev_phase/subtask2/dev/eng.csv')
labels = []
probs_list = []
labels = []
for text in tqdm(test_dataset['text']):
    # Run the model
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.sigmoid(logits).cpu().numpy()[0]
        pred_label = (probabilities > 0.5).astype(int)
        labels.append(pred_label)
        probs_list.append(probabilities)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 169/169 [00:02<00:00, 81.45it/s] 


In [ ]:
# print the results row by row in csv format
for i in range(len(labels)):
    print(f"{test_dataset['id'][i]}," + ",".join(str(x) for x in labels[i]))


arb_67be47e5216d7bee41e17484e619f4e6,0,0,0,0,1
arb_272322e5b265e177613d685e5619e402,0,0,0,0,0
arb_d1ec38dd0ec5d7a4fe28ef8317fc96c1,0,0,0,0,0
arb_fad75310b17c124d98ebc514189ec033,1,0,0,1,1
arb_95caf70cec5bf00c94c35cf7af2a0ab5,0,1,0,1,0
arb_ac108c1ecf5071892c61abd253847b15,1,0,0,0,1
arb_adaaa6d482119e65ce337ee224674e70,0,1,0,1,0
arb_2794b08cac6cc9394a68c51cfc436243,1,0,0,0,0
arb_19dd96c989323c9e950a2c3ab9c285be,0,0,0,0,0
arb_f2bd638d9d9fc7a617130ff2b198b562,0,1,0,1,0
arb_f992bf7776b854d4f7f8475aebf80f49,0,1,1,0,0
arb_0b5ac70e86926f5e84cad94028864a37,0,0,0,0,1
arb_8ababf95f952e2425c2df1033192dac0,0,0,0,0,0
arb_06cd19aac6cc52e394a22d7d1dd58efc,0,0,0,0,0
arb_12eeeb8d2fa2d04be2ed9830d5f36ce9,0,0,0,0,0
arb_5bc23bacf9a161cd0f99719c70681a81,0,0,0,0,1
arb_9ee7c931ab1ecd655533042d8301f6bb,0,0,0,0,0
arb_bb7c40559f3a7ca1ecdd7dd7c136198f,0,0,0,0,0
arb_5d394c0cce56675e2fc36a0590b47ed7,0,0,0,0,0
arb_0704305e8313650e672563a2d073384f,0,0,0,0,0
arb_e56b759d14fd70506e01cf971315453d,0,0,0,0,0
arb_d286a2aac

# Subtask 3: Manifestation Identification
Multi-label classification to classify how polarization is expressed, with multiple possible labels including Vilification, Extreme Language, Stereotype, Invalidation, Lack of Empathy, and Dehumanization.



In [ ]:
# Load the data
data = pd.read_csv('./dev_phase/subtask3/train/eng.csv')

# Split into train (80%) and val (20%), stratify if possible
train, val = train_test_split(
    data,
    test_size=0.2,
    random_state=42,
)

In [ ]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length # Store max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding=False, max_length=self.max_length, return_tensors='pt')

        # Ensure consistent tensor conversion for all items
        item = {key: encoding[key].squeeze() for key in encoding.keys()}
        # CHANGE THIS LINE: Use torch.float instead of torch.long for multi-label classification
        item['labels'] = torch.tensor(label, dtype=torch.float)
        return item

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

# Create train and Test dataset for multilabel
train_dataset = PolarizationDataset(train['text'].tolist(), train[['vilification','extreme_language','stereotype','invalidation','lack_of_empathy','dehumanization']].values.tolist(), tokenizer)
val_dataset = PolarizationDataset(val['text'].tolist(), val[['vilification','extreme_language','stereotype','invalidation','lack_of_empathy','dehumanization']].values.tolist(), tokenizer)

In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=6, problem_type="multi_label_classification") # use 6 labels

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=f"./",
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=100,
    disable_tqdm=False
)

# Define metrics function for multi-label classification
def compute_metrics_multilabel(p):
    # Sigmoid the predictions to get probabilities
    probs = torch.sigmoid(torch.from_numpy(p.predictions))
    # Convert probabilities to predicted labels (0 or 1)
    preds = (probs > 0.5).int().numpy()
    # Compute macro F1 score
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_multilabel,  # Use the new metrics function
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Macro F1 score on validation set for Subtask 3: {eval_results['eval_f1_macro']}")

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.
/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.369500,0.375278,0.483522
2,0.301000,0.357675,0.511258
3,0.254900,0.364251,0.486960


/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/0ssamaak0/miniconda3/envs/nlp/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Macro F1 score on validation set for Subtask 3: 0.48695961263769266


In [ ]:
from tqdm import tqdm

In [ ]:
test_dataset = pd.read_csv('./dev_phase/subtask3/dev/eng.csv')
labels = []
probs_list = []
labels = []
for text in tqdm(test_dataset['text']):
    # Run the model
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.sigmoid(logits).cpu().numpy()[0]
        pred_label = (probabilities > 0.5).astype(int)
        labels.append(pred_label)
        probs_list.append(probabilities)

100%|██████████| 169/169 [00:01<00:00, 125.50it/s]


In [ ]:
# print the results row by row in csv format
for i in range(len(labels)):
    print(f"{test_dataset['id'][i]}," + ",".join(str(x) for x in labels[i]))


arb_67be47e5216d7bee41e17484e619f4e6,0,0,0,0,0,0
arb_272322e5b265e177613d685e5619e402,0,0,0,0,0,0
arb_d1ec38dd0ec5d7a4fe28ef8317fc96c1,0,0,0,0,0,0
arb_fad75310b17c124d98ebc514189ec033,1,1,1,0,1,0
arb_95caf70cec5bf00c94c35cf7af2a0ab5,1,1,1,0,0,0
arb_ac108c1ecf5071892c61abd253847b15,1,0,0,0,0,0
arb_adaaa6d482119e65ce337ee224674e70,1,1,1,0,0,0
arb_2794b08cac6cc9394a68c51cfc436243,1,1,1,0,0,0
arb_19dd96c989323c9e950a2c3ab9c285be,1,1,1,0,0,0
arb_f2bd638d9d9fc7a617130ff2b198b562,1,1,1,0,0,0
arb_f992bf7776b854d4f7f8475aebf80f49,1,1,1,0,1,0
arb_0b5ac70e86926f5e84cad94028864a37,1,0,0,0,0,0
arb_8ababf95f952e2425c2df1033192dac0,0,0,0,0,0,0
arb_06cd19aac6cc52e394a22d7d1dd58efc,0,0,0,0,0,0
arb_12eeeb8d2fa2d04be2ed9830d5f36ce9,0,0,0,0,0,0
arb_5bc23bacf9a161cd0f99719c70681a81,1,0,0,0,0,0
arb_9ee7c931ab1ecd655533042d8301f6bb,0,0,0,0,0,0
arb_bb7c40559f3a7ca1ecdd7dd7c136198f,0,0,0,0,0,0
arb_5d394c0cce56675e2fc36a0590b47ed7,0,0,0,0,0,0
arb_0704305e8313650e672563a2d073384f,0,0,0,0,0,0
arb_e56b759d14fd7050